In [30]:
import pandas as pd
df = pd.read_csv('../csv_data/demesne_v2.csv')
df

,TEXT,X1,Y1,X2,Y2,block_no,line_no
0,CASH,383,20,427,42,0,0
1,SALES,431,20,482,42,0,0
2,INVOICE,486,20,557,42,0,0
3,DEMESNE,19,21,78,38,0,1
4,ELECTRICAL,81,21,156,38,0,1
...,...,...,...,...,...,...,...
244,Duncrue,416,766,450,777,59,1
245,Street,453,766,478,777,59,1
246,Belfast,404,776,432,787,59,2
247,BT3,434,776,450,787,59,2


In [35]:
df_sort= df.sort_values(by=['Y1','X1']).reset_index(drop=True)


In [36]:
pos = find_text(df,'VAT')
df[pos[1]:pos[1]+5]

,TEXT,X1,Y1,X2,Y2,block_no,line_no
118,VAT,77,529,93,540,34,0
119,Analysis,96,529,129,540,34,0
120,Rate,33,539,52,550,35,0
121,Net,84,539,98,550,35,1
122,Total,101,539,121,550,35,1


In [18]:
df_sort= df.sort_values(by=['Y1','X1']).reset_index(drop=True)


In [37]:
def rearrange(df):
    for i in range(1, len(df)):
        temp=df.loc[i,'Y1'] - df.loc[i-1, 'Y1']
        if -5 < temp < 10:
            df.loc[i,'Y1'] = df.loc[i-1,'Y1']
    
    return df.sort_values(by=['Y1','X1']).reset_index(drop=True)
    

In [38]:
df = rearrange(df_sort)

In [39]:
def find_text(temp_df, text,case=False):
    location = temp_df[temp_df['TEXT'].str.contains(text,case=case, na=False)].index.tolist()
    return location

def find_next_word(temp_df, text, nearest_text, case=False):
    pos = find_text(temp_df,text,case=case)
    pos = [item +1 for item in pos]
#     print(pos)
    new_pos = list()
    for item in pos:
#         temp = df.loc[]
        temp_value= temp_df.loc[item]['TEXT']
#         print(temp_value.lower())
       
        if temp_value.lower().__contains__(nearest_text.lower()):
            new_pos.append(item-1)
        
    return new_pos

In [40]:
pos = find_text(df,'Invoice')

df[pos[2]+2:pos[2]+4]['TEXT'].values[0]
# invoice_no = df[pos[0]+2:pos[0]+3]['TEXT'].values[0]

'998114'

In [41]:
pos_date =find_text(df, 'date')[0]
date = df[pos_date+1:pos_date+2]['TEXT'].values[0]
date

'07/04/2020'

In [42]:
pos = find_text(df,'date')

In [43]:
pos = find_text(df,'VAT')
df[pos[0]:pos[0]+5]

,TEXT,X1,Y1,X2,Y2,block_no,line_no
111,VAT,77,529,93,540,34,0
112,Analysis,96,529,129,540,34,0
113,Rate,33,539,52,550,35,0
114,Net,84,539,98,550,35,1
115,Total,101,539,121,550,35,1


In [44]:
find_next_word(df,'VAT','Analysis')

[111]

In [45]:
table_start = find_text(df, 'Unit')
print(table_start)
table_end = find_next_word(df,'VAT','Analysis')
print(table_end)
table_data = df[table_start[0]:table_end[0]+10]
df[95:107]

[94, 227, 234]
[111]


,TEXT,X1,Y1,X2,Y2,block_no,line_no
95,Price,424,328,445,339,16,3
96,Amount,518,328,550,339,16,4
97,8,50,349,55,360,27,0
98,T1EN60X100G,82,349,133,361,27,1
99,Panel,188,349,208,360,27,2
100,Trunking,210,349,241,360,27,2
101,60X100,244,349,271,360,27,2
102,G,273,349,278,360,27,2
103,Narrow,281,349,307,360,27,2
104,Slot,309,349,323,360,27,2


In [46]:
from pprint import pprint
msg = dict()
result = list()
for index, row in table_data.iterrows():
    
    if row['X1'] < 60 and row['TEXT'].isdigit(): 
#         print(row['TEXT'])
        msg = dict()
        msg['qty'] = row['TEXT']
        msg['part'] = ''
    
    if ('qty' in msg) and (60 < row['X1'] < 90):
#         print(row['TEXT'])
        msg['part'] = row['TEXT']
        msg['desc'] = ''
   
    if ('desc' in msg) and (180 < row['X1'] < 360):
#         print(row['TEXT'])
        msg['desc'] = msg['desc'] +" "+ row['TEXT']
        msg['unit'] = ''
   
    if ('unit' in msg) and (400  < row['X1'] < 460):
#         print(row['TEXT'])
        msg['unit'] = row['TEXT']
        msg['disc'] = 0
        msg['total'] = 0 
        
    if 'disc' in msg and 470  < row['X1'] < 500:
        msg['disc'] = row['TEXT']
        msg['total'] = 0
        
    if 'total' in msg and 520 < row['X1'] < 560:
        
        msg['total'] = row['TEXT']
        pprint(msg)
        result.append(msg)
        msg = dict()
    

{'desc': ' Panel Trunking 60X100 G Narrow Slot 2 Mt 2585',
 'disc': '25.00%',
 'part': 'T1EN60X100G',
 'qty': '8',
 'total': '111.53',
 'unit': '18.58831'}


In [48]:
result

[{'qty': '8',
  'part': 'T1EN60X100G',
  'desc': ' Panel Trunking 60X100 G Narrow Slot 2 Mt 2585',
  'unit': '18.58831',
  'disc': '25.00%',
  'total': '111.53'}]

In [118]:
pos_good = find_text(df, 'Goods', case=True)
pos_good

[]

In [12]:
pos_vat = find_text(df, 'V.A.T', case=True)
pos_vat

[247]

In [32]:
good_value = df[pos_good[1]+1:pos_good[1]+2]['TEXT'].values[0]

In [33]:
vat_value = df[pos_vat[1]+1:pos_vat[1]+2]['TEXT'].values[0]

In [34]:
vat_value

'€327.94'

In [38]:
pos_total = find_text(df,'Total', True)[0]
pos_total

130

In [43]:
total_value = df[pos_total+1:pos_total+2]['TEXT'].values[0]

In [44]:
total_value

'€1753.76'

,TEXT,X1,Y1,X2,Y2,block_no,line_no
131,€1753.76,512,605,552,616,21,1
